### Task 1

#### Levenstein distance

Let's implement Levenstein distance between two string. Replacement cost is 2.

In [1]:
def lev(a, b):
    if len(a) == 0:
        return len(b)
    elif len(b) == 0:
        return len(a)
    elif a[0] == b[0]:
        return lev(a[1:], b[1:])
    else:
        return 1 + min(lev(a[1:], b), lev(a, b[1:]), 1 + lev(a[1:], b[1:]))

In [2]:
lev("home", "horse")

3

#### QWERTY distance modification

Now, we can modify the algorithm so the substitution cost depends on the distance on QWERTY keyboard

We will have the cost of substitution depend on the quartile of the distance. Lets calculate them

In [12]:
import pandas as pd
qwerty_dist = pd.read_csv("keys.csv", sep = ";").astype({"Distance": "float"})
qwerty_dist

,Key1,Key2,Distance
0,A,A,0.00
1,A,B,87.82
2,A,C,51.29
3,A,D,38.10
4,A,E,38.40
...,...,...,...
671,Z,V,57.15
672,Z,W,38.40
673,Z,X,19.05
674,Z,Y,89.48


In [15]:
desc = qwerty_dist["Distance"].describe()
desc

count    676.000000
mean      62.101302
std       37.884516
min        0.000000
25%       34.340000
50%       56.445000
75%       87.820000
max      171.450000
Name: Distance, dtype: float64

So now basically if the distance is < 34 (25%) the replacement cost will be 1, if < 56 (50%) the cost will be 2 and so on

Let's calculate the cost

In [17]:
q1 = qwerty_dist["Distance"].quantile(0.25)
q2 = qwerty_dist["Distance"].quantile(0.5)
q3 = qwerty_dist["Distance"].quantile(0.75)
qwerty_dist["Cost"] = qwerty_dist["Distance"].map(lambda dist: 1 if dist < q1 else 2 if dist < q2 else 3 if dist < q3 else 4)
qwerty_dist

,Key1,Key2,Distance,Cost
0,A,A,0.00,1
1,A,B,87.82,4
2,A,C,51.29,2
3,A,D,38.10,2
4,A,E,38.40,2
...,...,...,...,...
671,Z,V,57.15,3
672,Z,W,38.40,2
673,Z,X,19.05,1
674,Z,Y,89.48,4


Finally, let's implement the function

In [52]:
def qwerty_lev(a, b):
    if len(a) == 0:
        return len(b)
    elif len(b) == 0:
        return len(a)
    elif a[0] == b[0]:
        return qwerty_lev(a[1:], b[1:])
    else:
        return min(qwerty_lev(a[1:], b) + 1, qwerty_lev(a, b[1:]) + 1, qwerty_dist["Cost"][(qwerty_dist["Key1"] == a[0].capitalize()) & (qwerty_dist["Key2"] == b[0].capitalize())].iloc[0] + qwerty_lev(a[1:], b[1:]))

In [53]:
qwerty_lev("home", "jome")

1

In [54]:
qwerty_lev("home", "kome")

2

#### Transposition modification

In [55]:
def transpos_lev(a, b):
    if len(a) == 0:
        return len(b)
    elif len(b) == 0:
        return len(a)
    elif a[0] == b[0]:
        return transpos_lev(a[1:], b[1:])
    elif len(a) > 1 and len(b) > 1 and a[0] == b[1] and a[1] == b[0]:
        return 1 + min(transpos_lev(a[1:], b), transpos_lev(a, b[1:]), 1 + transpos_lev(a[1:], b[1:]), transpos_lev(a[2:], b[2:]))
    else:
        return 1 + min(transpos_lev(a[1:], b), transpos_lev(a, b[1:]), 1 + transpos_lev(a[1:], b[1:]))

In [57]:
transpos_lev("home", "hoem")

1

### Task 2

Let's start with obtaining vocabulary from first 3 texts of nltk

In [77]:
from nltk.book import *
def get_vocabulary(texts):
    voc = []
    for text in texts:
        voc.extend(text.tokens)
    voc = sorted(list(set(voc)))
    return voc
vocabulary = get_vocabulary([text1, text2, text3])
vocabulary = vocabulary[vocabulary.index("A"):]
vocabulary

['A',
 'ABOUT',
 'ACCOUNT',
 'ADDITIONAL',
 'ADVANCING',
 'ADVENTURES',
 'AFFGHANISTAN',
 'AFRICA',
 'AFTER',
 'AGAINST',
 'AHAB',
 'ALFRED',
 'ALGERINE',
 'ALIVE',
 'ALL',
 'ALMOST',
 'ALONE',
 'ALWAYS',
 'AM',
 'AMERICA',
 'AMONG',
 'ANCHORS',
 'AND',
 'ANGLO',
 'ANIMAL',
 'ANNALS',
 'ANNUS',
 'ANOTHER',
 'ANY',
 'APOLOGY',
 'APPLICATION',
 'APPROACHING',
 'ARCTIC',
 'ARE',
 'AROUND',
 'AS',
 'ASCENDING',
 'ASIA',
 'ASIDE',
 'ASPECT',
 'AT',
 'ATTACK',
 'ATTACKED',
 'ATTITUDES',
 'AUGUST',
 'AUTHOR',
 'AZORE',
 'Abashed',
 'Abbeyland',
 'Abednego',
 'Abel',
 'Abelmizraim',
 'Abidah',
 'Abide',
 'Abimael',
 'Abimelech',
 'Abjectus',
 'Aboard',
 'Abominable',
 'About',
 'Above',
 'Abr',
 'Abrah',
 'Abraham',
 'Abram',
 'Absence',
 'Abundance',
 'Academy',
 'Accad',
 'Accessory',
 'According',
 'Accordingly',
 'Accursed',
 'Achbor',
 'Achilles',
 'Actium',
 'Acushnet',
 'Adah',
 'Adam',
 'Adbeel',
 'Add',
 'Adieu',
 'Adios',
 'Admah',
 'Admiral',
 'Admirals',
 'Adullamite',
 'Advance',


Now let's calculate the times word t occurs in the text d.

In [80]:
from nltk import FreqDist
def count(word, document):
    fdist = FreqDist(document)
    if word in fdist.keys:
        return fdist[word]
    return 0
count("Moby", text1)

84

Now, create a vector of tex1 and tex2

In [ ]:
def vectorize_old(text, voc):
    vect = []
    for word in voc:
        vect.append(count(word, text))
    return vect
vectorize(text1, vocabulary)

Such an approach works really slow. So lets try counting words simultaniously.

In [84]:
def vectorize(text, voc):
    vect = [0 for i in range(len(voc))]
    for word in text1:
        if word >= "A":
            vect[voc.index(word)] += 1
    return vect
vectorize(text1, vocabulary)

[167,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 9,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 37,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 5,
 2,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 1,
 6,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 6,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 7,
 3,
 1,
 1,
 17,
 1,
 9,
 1,
 1,
 1,
 18,
 501,
 1,
 2,
 1,
 0,
 2,
 0,
 3,
 2,
 0,
 0,
 1,
 6,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 55,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 9,
 9,
 2,
 3,
 4,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 1,
 2,
 11,
 34,
 6,
 1,
 1,
 0,
 8,
 0,
 0,
 0,
 0,
 3,
 14,
 1,
 0,
 1,
 0,
 1,
 369,
 7,
 1,
 4,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 8,
 2,
 5,
 3,
 1,
 1,
 1,
 1,
 0,
 6,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 3,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 1,
 2,
 1,
 5,
 4,
 0,
 11,
 0,
 1,
 1,
 3,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 

Finally, let's calculate distance between text1 and text2

In [90]:
import numpy as np
text1_vect = np.array(vectorize(text1, vocabulary))
text2_vect = np.array(vectorize(text2, vocabulary))
dist = np.dot(text1_vect,text2_vect)
print("The distance between text1 and text2: ", dist)
print("The cosine distance between text1 and text2: ", dist / (len(vocabulary)**2))


The distance between text1 and text2:  405369599
The cosine distance between text1 and text2:  0.8269070820725684


We calculated the cosine distance, because the first number doesn't give us any real idea about the distance. The value of 0.82 indicates that texts are quite similar, because it is close to 1.